In [ ]:
import numpy as np

input_params_keys = {
    "MHL": "lightest Higgs mass",
    "MHH": "heavy Higgs mass",
    "MHA": "pseudo-scalar Higgs mass",
    "MHp": "charged Higgs mass",
    "lam5": "Split..",
    "cba": "cos(beta-alpha)",
    "tb": "tan(beta)",
    "vev": "vacuum expectation value",
}

In [ ]:
from src.thdm_param_scan import calculate_random_sampling, check_vacuum_conditions, STUCalculator

In [ ]:
higgs_mass = 125.35  # Higgs mass in GeV
vev = 246.22  # Vacuum expectation value in GeV

mass_range = (0, 8 * higgs_mass)

epsilon_range = (1e-12, 1e-1)

delta_range = (-0.10, 0.10)

n_points = int(1e6)

results_df = calculate_random_sampling(mass_range, epsilon_range, delta_range, n_samples=n_points)
results_with_conditions = check_vacuum_conditions(results_df, verbose=True)

In [ ]:
# Check current columns in results_with_conditions
print("Current columns in results_with_conditions:")
print(results_with_conditions.columns.tolist())
print(f"\nNumber of rows: {len(results_with_conditions)}")

In [ ]:
n_sigma = 2  # Number of sigma for STU calculation

results_with_conditions["S"] = np.nan
results_with_conditions["T"] = np.nan
results_with_conditions["U"] = np.nan
results_with_conditions["STU_constraint"] = False
results_with_conditions["S_constraint"] = False
results_with_conditions["T_constraint"] = False
results_with_conditions["U_constraint"] = False
results_with_conditions["n_sigma_used"] = n_sigma

In [ ]:
stu_calculator = STUCalculator()

# Solo calcular STU para las filas donde all_conditions_satisfied es True
if "all_conditions_satisfied" in results_with_conditions.columns:
    # Crear una máscara para las filas que cumplen las condiciones
    mask = results_with_conditions["all_conditions_satisfied"] == True

    # Solo aplicar el cálculo a las filas que cumplen las condiciones
    if mask.any():
        print(
            f"Calculando STU para {mask.sum()} filas que cumplen all_conditions=True de un total de {len(results_with_conditions)} filas"
        )

        # Crear una copia temporal solo con las filas que cumplen las condiciones
        temp_df = results_with_conditions[mask].copy()

        # Calcular STU solo para estas filas
        temp_df = stu_calculator.calculate_STU_dataframe(temp_df, verbose=True, n_sigma=n_sigma)

        # Actualizar las columnas STU en el DataFrame original solo para las filas que cumplen las condiciones
        stu_columns = [
            "S",
            "T",
            "U",
            "STU_constraint",
            "S_constraint",
            "T_constraint",
            "U_constraint",
        ]
        for col in stu_columns:
            if col in temp_df.columns:
                results_with_conditions.loc[mask, col] = temp_df[col].values

        print(f"STU calculado exitosamente para {mask.sum()} filas")
    else:
        print("No hay filas que cumplan all_conditions=True")
else:
    print("Advertencia: La columna 'all_conditions' no existe en el DataFrame")
    print("Aplicando cálculo STU a todo el DataFrame como respaldo")
    results_with_conditions = stu_calculator.calculate_STU_dataframe(
        results_with_conditions, verbose=True, n_sigma=n_sigma
    )

results_with_conditions.to_feather("data/results_with_conditions.feather", index=False)